In [1]:
from bs4 import BeautifulSoup
import sys #저장 시 사용하는 모듈
import pandas as pd
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

#driver = webdriver.Chrome('chromedrive|r', options=options) #창안뜨게 옵션 적용

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
#크롬 드라이버를 사용해서 웹 브라우저 실행
path = "/content/drive/MyDrive/Colab Notebooks/chromedriver.exe"

driver = webdriver.Chrome('chromedriver')
web="https://eminwon.icjg.go.kr/emwp/gov/mogaha/ntis/web/emwp/cmmpotal/action/EmwpMainMgtAction.do"

driver.get(web)
time.sleep(1) #인터넷 창이 모두 열릴 때까지 2초 기다림

minwon_df = []
df = pd.DataFrame(minwon_df, columns = ['page','num','title','date','answer'])
num = 0


In [4]:
#온라인 민원상담 페이지로 이동
driver.find_element(By.XPATH, r'//*[@id="header"]/div[2]/div/div[2]/ul/li[1]/a').click()
time.sleep(1) 
driver.find_element(By.XPATH,'//*[@id="allmenu"]/div/div[2]/div[1]/ul/li/a').click()

In [5]:
#셀레니움으로 민원목록 가져오기
for page in range(77): #가져오는 페이지
    html = driver.page_source  #현재 화면의 html을 모두 불러옴
    soup = BeautifulSoup(html, 'html.parser')  #불러오는 정보를 걸러주는 도구
    
    minwon_dates = [ date.get_text() for date in soup.find_all('td', attrs={'class':'td-date'})] # 민원 작성일
    minwon_num=[num.get_text()for num in soup.find_all('td',attrs={'class':'td-num'})] # 민원 번호
    minwon_answer=[answer.get_text()for answer in soup.find_all('td',attrs={'class':'td-answer'})] #답변여부
    
    minwon_date = [] #민원 작성일 정제
    for date in minwon_dates:
        if re.search(r'\d+.\d+.\d+.', date) != None: 
            date = re.sub("-",".",date)
            minwon_date.append(date)
            
    minwonlist = soup.select("table > tbody > tr > td.td-list > a")
    links=driver.find_elements(By.CSS_SELECTOR,"table > tbody > tr > td.td-list > a")
    n=-1
    for i in minwonlist:
        n=n+1
        minwon = i.text.strip()
        
        df = df.append({'page':page+1,'num':minwon_num[n],'date':minwon_date[n],'title':minwon,'answer':minwon_answer[n]}, ignore_index=True)
     
    if page > 100: 
        element = driver.find_element(By.XPATH,'/html/body/main/div/form/div[2]/a[11]')
    else: element = driver.find_element(By.XPATH,'/html/body/main/div/form/div[2]/a[12]')
    driver.execute_script("arguments[0].click();", element)
    time.sleep(2)

minwon_df = df  

In [7]:
#pd.set_option('display.max_rows', None)
minwonlist=minwon_df

In [30]:
minwonlist.to_csv("minwonlist.csv", mode='w')

In [6]:
"""
#답변완료된 df
answer_df=minwon_df[minwon_df['answer']=='답변완료']
answer_df
"""

"\n#답변완료된 df\nanswer_df=minwon_df[minwon_df['answer']=='답변완료']\nanswer_df\n"

In [31]:
"""
import requests
from bs4 import BeautifulSoup

# 검색 조건 설정
payload = {
    "searchStartDt": "2022-01-01",
    "searchEndDt": "2022-12-31",
    "currentPage": "1",
    "pagePerRow": "100",
    "searchWord": "",
    "searchType": "wrcTitle"
}

# 공개민원내역 페이지 URL
url = "https://eminwon.icjg.go.kr/emwp/gov/mogaha/ntis/web/emwp/cmmpotal/action/EmwpMainMgtAction.do"

# POST 요청 보내기
response = requests.post(url, data=payload)

# HTML 파싱
soup = BeautifulSoup(response.content, "html.parser")

# 민원내역 추출
table = soup.find("table", {"class": "boardList"})
rows = table.find_all("tr")[1:]  # 첫 번째 행은 헤더이므로 제외
for row in rows:
    columns = row.find_all("td")
    no = columns[0].text.strip()
    title = columns[1].text.strip()
    name = columns[2].text.strip()
    date = columns[3].text.strip()
    print(f"{no}\t{title}\t{name}\t{date}")
"""

'\nimport requests\nfrom bs4 import BeautifulSoup\n\n# 검색 조건 설정\npayload = {\n    "searchStartDt": "2022-01-01",\n    "searchEndDt": "2022-12-31",\n    "currentPage": "1",\n    "pagePerRow": "100",\n    "searchWord": "",\n    "searchType": "wrcTitle"\n}\n\n# 공개민원내역 페이지 URL\nurl = "https://eminwon.icjg.go.kr/emwp/gov/mogaha/ntis/web/emwp/cmmpotal/action/EmwpMainMgtAction.do"\n\n# POST 요청 보내기\nresponse = requests.post(url, data=payload)\n\n# HTML 파싱\nsoup = BeautifulSoup(response.content, "html.parser")\n\n# 민원내역 추출\ntable = soup.find("table", {"class": "boardList"})\nrows = table.find_all("tr")[1:]  # 첫 번째 행은 헤더이므로 제외\nfor row in rows:\n    columns = row.find_all("td")\n    no = columns[0].text.strip()\n    title = columns[1].text.strip()\n    name = columns[2].text.strip()\n    date = columns[3].text.strip()\n    print(f"{no}\t{title}\t{name}\t{date}")\n'